7.1 可接受任意数量参数的函数

问题

你想构造一个可接受任意数量参数的函数。

解决方案

为了能让一个函数接受任意数量的位置参数，可以使用一个*参数。例如：



In [2]:
def avg(first,*rest):
    return (first+sum(rest))/(1+len(rest))

In [3]:
avg(1,2)

1.5

In [4]:
avg(1,2,3,4)

2.5

In [5]:
import html

def make_element(name, value, **attrs):
    keyvals = [' %s="%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element = '<{name}{attrs}>{value}</{name}>'.format(
                name=name,
                attrs=attr_str,
                value=html.escape(value))
    return element

# Example
# Creates '<item size="large" quantity="6">Albatross</item>'
make_element('item', 'Albatross', size='large', quantity=6)

# Creates '<p>&lt;spam&gt;</p>'
make_element('p', '<spam>')

'<p>&lt;spam&gt;</p>'

如果你还希望某个函数能同时接受任意数量的位置参数和关键字参数，可以同时使用*和**。比如：

In [6]:
def anyargs(*args,**kwargs):
    print(args)#tuple
    print(**kwargs)#dict
#使用这个函数时，所有位置参数会被放到args元组中，所有关键字参数会被放到字典kwargs中。

一个*参数只能出现在函数定义中最后一个位置参数后面，而 **参数只能出现在最后一个参数。 有一点要注意的是，在*参数后面仍然可以定义其他参数。



In [7]:
def a(x, *args, y):
    pass

def b(x, *args, y, **kwargs):
    pass


7.2 只接受关键字参数的函数

问题

你希望函数的某些参数强制使用关键字参数传递

解决方案

将强制关键字参数放到某个*参数或者单个*后面就能达到这种效果。比如：



In [9]:
def recv(maxsize,*,block):
    pass
recv(1024,True)

TypeError: recv() takes 1 positional argument but 2 were given

In [10]:
recv(1024,block=True)

In [11]:
#利用这种技术，我们还能在接受任意多个位置参数的函数中指定关键字参数。比如：
def minimum(*values,clip=None):
    m = min(values)
    if clip is not None:
        m = clip if clip > m else m
    return m
minimum(1, 5, 2, -5, 10)


-5

In [12]:
minimum(1, 5, 2, -5, 10, clip=0) # Returns 0

0

7.3 给函数参数增加元信息

问题

你写好了一个函数，然后想为这个函数的参数增加一些额外的信息，这样的话其他使用者就能清楚的知道这个函数应该怎么使用。

解决方案

使用函数参数注解是一个很好的办法，它能提示程序员应该怎样正确使用这个函数。 例如，下面有一个被注解了的函数：



In [13]:
def add(x:int, y:int) -> int:
    return  x +y

python解释器不会对这些注解添加任何的语义。它们不会被类型检查，运行时跟没有加注解之前的效果也没有任何差距。 然而，对于那些阅读源码的人来讲就很有帮助啦。第三方工具和框架可能会对这些注解添加语义。同时它们也会出现在文档中。



In [14]:
help(add)

Help on function add in module __main__:

add(x:int, y:int) -> int



尽管你可以使用任意类型的对象给函数添加注解(例如数字，字符串，对象实例等等)，不过通常来讲使用类或者字符串会比较好点。



In [15]:
#函数注解只存储在函数的 __annotations__ 属性中。例如：

add.__annotations__

{'x': int, 'y': int, 'return': int}

尽管注解的使用方法可能有很多种，但是它们的主要用途还是文档。 因为python并没有类型声明，通常来讲仅仅通过阅读源码很难知道应该传递什么样的参数给这个函数。 这时候使用注解就能给程序员更多的提示，让他们可以正确的使用函数。



In [16]:
b = 1,2
b

(1, 2)

In [17]:
def spam(a,b=42):
    print(a,b)
    
spam(1,2)

1 2


如果默认参数是一个可修改的容器比如一个列表、集合或者字典，可以使用None作为默认值，就像下面这样：

In [18]:
def spam(a,b=None):
    if b is None:
        b =[]

In [24]:
#如果你并不想提供一个默认值，而是想仅仅测试下某个默认参数是不是有传递进来，可以像下面这样写
_no_value= object()
def spam(a,b=_no_value):
    if b is _no_value:
        print('No b value supploed')

#测试一下函数
# spam(1)
# spam(1,2)
spam(1,None)

In [25]:
x = 10
a = lambda y : x+y
x =20
b = lambda y: x+y

In [26]:
print(a(10),b(10))
'''
这其中的奥妙在于lambda表达式中的x是一个自由变量，
在运行时绑定值，而不是定义时就绑定，这跟函数的默认值参数定义是不同的。
因此，在调用这个lambda表达式的时候，x的值是执行时的值。例如：
'''

30 30


In [27]:
x = 15
a(10)

25

In [28]:
x =3
a(10)

13

In [30]:
#如果你想让某个匿名函数在定义时就捕获到值，可以将那个参数值定义成默认参数即可，就像下面这样：
x =10
a = lambda y,x=x:x+y
x = 20
b = lambda y,x=x:x+y
print(a(10),b(10))

20 30


在这里列出来的问题是新手很容易犯的错误，有些新手可能会不恰当的使用lambda表达式。 比如，通过在一个循环或列表推导中创建一个lambda表达式列表，并期望函数能在定义时就记住每次的迭代值。例如：



In [31]:
func = [lambda x: x+n for n in range(5)]
for f in func:
    print(f(0))

4
4
4
4
4


In [33]:
#但是实际效果是运行是n的值为迭代的最后一个值。现在我们用另一种方式修改一下：

func = [lambda x,n=n: x+n for n in range(5)]
for f in func:
    print(f(0))

0
1
2
3
4


7.8 减少可调用对象的参数个数

问题

你有一个被其他python代码使用的callable对象，可能是一个回调函数或者是一个处理器， 但是它的参数太多了，导致调用时出错。

解决方案

如果需要减少某个函数的参数个数，你可以使用 functools.partial() 。 partial() 函数允许你给一个或多个参数设置固定的值，减少接下来被调用时的参数个数。 为了演示清楚，假设你有下面这样的函数：

In [34]:
def spam(a,s,d,f):
    print(a,s,d,f)

In [37]:
from functools import partial
s1 = partial(spam,1)
s1(2,3,4)

1 2 3 4


可以看出 partial() 固定某些参数并返回一个新的callable对象。这个新的callable接受未赋值的参数， 然后跟之前已经赋值过的参数合并起来，最后将所有参数传递给原始函数。

本节要解决的问题是让原本不兼容的代码可以一起工作。下面我会列举一系列的例子。

第一个例子是，假设你有一个点的列表来表示(x,y)坐标元组。 你可以使用下面的函数来计算两点之间的距离：


In [38]:
points = [ (1, 2), (3, 4), (5, 6), (7, 8) ]

import math
def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.hypot(x2 - x1, y2 - y1)

现在假设你想以某个点为基点，根据点和基点之间的距离来排序所有的这些点。 列表的 sort() 方法接受一个关键字参数来自定义排序逻辑， 但是它只能接受一个单个参数的函数(distance()很明显是不符合条件的)。 现在我们可以通过使用 partial() 来解决这个问题：



In [39]:
pt = (4,3)
points.sort(key=partial(distance,pt))
points

[(3, 4), (1, 2), (5, 6), (7, 8)]

更进一步，partial() 通常被用来微调其他库函数所使用的回调函数的参数。 例如，下面是一段代码，使用 multiprocessing 来异步计算一个结果值， 然后这个值被传递给一个接受一个result值和一个可选logging参数的回调函数：



In [40]:
def output_result(result, log=None):
    if log is not None:
        log.debug('Got: %r', result)

# A sample function
def add(x, y):
    return x + y

if __name__ == '__main__':
    import logging
    from multiprocessing import Pool
    from functools import partial

    logging.basicConfig(level=logging.DEBUG)
    log = logging.getLogger('test')

    p = Pool()
    p.apply_async(add, (3, 4), callback=partial(output_result, log=log))
    p.close()
    p.join()

DEBUG:test:Got: 7


当给 apply_async() 提供回调函数时，通过使用 partial() 传递额外的 logging 参数。 而 multiprocessing 对这些一无所知——它仅仅只是使用单个值来调用回调函数。
作为一个类似的例子，考虑下编写网络服务器的问题，socketserver 模块让它变得很容易。 下面是个简单的echo服务器：




In [41]:
from socketserver import StreamRequestHandler, TCPServer

class EchoHandler(StreamRequestHandler):
    def handle(self):
        for line in self.rfile:
            self.wfile.write(b'GOT:' + line)

serv = TCPServer(('', 15000), EchoHandler)
serv.serve_forever()

KeyboardInterrupt: 

In [42]:
from urllib.request import urlopen

class UrlTemplate:
    def __init__(self, template):
        self.template = template

    def open(self, **kwargs):
        return urlopen(self.template.format_map(kwargs))

# Example use. Download stock data from yahoo
yahoo = UrlTemplate('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')
for line in yahoo.open(names='IBM,AAPL,FB', fields='sl1c1v'):
    print(line.decode('utf-8'))

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [43]:
def urltemplate(template):
    def opener(**kwargs):
        return urlopen(template.format_map(kwargs))
    return opener

# Example use
yahoo = urltemplate('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')
for line in yahoo(names='IBM,AAPL,FB', fields='sl1c1v'):
    print(line.decode('utf-8'))

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [ ]:
def print_result(result):
    print('GOt:',result)
apply_    

7.12 访问闭包中定义的变量

问题

你想要扩展函数中的某个闭包，允许它能访问和修改函数的内部变量。

解决方案

通常来讲，闭包的内部变量对于外界来讲是完全隐藏的。 但是，你可以通过编写访问函数并将其作为函数属性绑定到闭包上来实现这个目的。例如：



In [51]:
def sample():
    n = 0
    def func():
        print('n=',n)
    def get_n():
        return n
    def set_n(value):
        nonlocal n
        n = value
    func.get_n = get_n
    func.get_n=set_n
    return func

In [52]:
f = sample()
f()

n= 0


In [53]:
f.set_n(10)
f.get_n()

AttributeError: 'function' object has no attribute 'set_n'

为了说明清楚它如何工作的，有两点需要解释一下。首先，nonlocal 声明可以让我们编写函数来修改内部变量的值。 其次，函数属性允许我们用一种很简单的方式将访问方法绑定到闭包函数上，这个跟实例方法很像(尽管并没有定义任何类)。

还可以进一步的扩展，让闭包模拟类的实例。你要做的仅仅是复制上面的内部函数到一个字典实例中并返回它即可。例如：



In [54]:
import sys
class ClosureInstance:
    def __init__(self, locals=None):
        if locals is None:
            locals = sys._getframe(1).f_locals

        # Update instance dictionary with callables
        self.__dict__.update((key,value) for key, value in locals.items()
                            if callable(value) )
    # Redirect special methods
    def __len__(self):
        return self.__dict__['__len__']()

# Example use
def Stack():
    items = []
    def push(item):
        items.append(item)

    def pop():
        return items.pop()

    def __len__():
        return len(items)

    return ClosureInstance()

In [55]:
#下面是一个交互式会话来演示它是如何工作的：

s = Stack()
s

In [57]:
s.push(10)
s.push(20)
s.push('Hello')
len(s)

4

In [58]:
s.pop()

'Hello'

In [59]:
s.pop()

20

In [60]:
s.pop()

10

In [61]:
s.pop()

10

有趣的是，这个代码运行起来会比一个普通的类定义要快很多。你可能会像下面这样测试它跟一个类的性能对比：



In [62]:
class Stack2:
    def __init__(self):
        self.items = []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        return self.items.pop()

    def __len__(self):
        return len(self.items)


In [63]:
from timeit import timeit
s = Stack()
timeit('s.push(1);s.pop()', 'from __main__ import s')

0.5658539620344527

In [64]:
s = Stack2()
timeit('s.push(1);s.pop()', 'from __main__ import s')

0.6213986150105484

结果显示，闭包的方案运行起来要快大概8%，大部分原因是因为对实例变量的简化访问， 闭包更快是因为不会涉及到额外的self变量。

Raymond Hettinger对于这个问题设计出了更加难以理解的改进方案。不过，你得考虑下是否真的需要在你代码中这样做， 而且它只是真实类的一个奇怪的替换而已，例如，类的主要特性如继承、属性、描述器或类方法都是不能用的。 并且你要做一些其他的工作才能让一些特殊方法生效(比如上面 ClosureInstance 中重写过的 __len__() 实现。)

最后，你可能还会让其他阅读你代码的人感到疑惑，为什么它看起来不像一个普通的类定义呢？ (当然，他们也想知道为什么它运行起来会更快)。尽管如此，这对于怎样访问闭包的内部变量也不失为一个有趣的例子。

总体上讲，在配置的时候给闭包添加方法会有更多的实用功能， 比如你需要重置内部状态、刷新缓冲区、清除缓存或其他的反馈机制的时候